# Understand Sampler Theory

## Introduction

The **Sampler** is one of the foundational primitives in Qiskit. Unlike the **Estimator**, which calculates expectation values of observables ($ \langle \psi | H | \psi \rangle $), the Sampler is designed to approximate the **quasi-probability distribution** of measurement outcomes.

In simple terms, if you run a quantum circuit that prepares a state $|\psi\rangle$, measuring it in the computational basis produces a bitstring (e.g., `101`). Running this many times (shots) gives you a histogram of counts. The Sampler formalizes this process but adds a layer of abstraction that allows for:

1.  **Error Mitigation**: Automatically acting to correct or suppress errors in the readout or execution.
2.  **Session Management**: Efficiently queuing multiple jobs (when using the cloud runtime).
3.  **Unified Interface**: Working consistently across simulators and real hardware.

This notebook covers the theory behind what the Sampler does, how it differs from a raw backend execution, and the theoretical underpinnings of the error suppression techniques it employs.

## 1. Sampler Concept: Probability Distributions

At its core, the Sampler estimates the probability $P(k)$ of measuring a basis state $|k\rangle$:

$$ P(k) = |\langle k | \psi \rangle|^2 $$

While a standard `backend.run()` returns raw counts (e.g., `{'00': 50, '11': 50}`), the Sampler often treats these results as samples from an underlying probability distribution. 

### Why "Quasi"-Probability?

You may hear the term "quasi-probability" (especially with Sampler V1 or certain error mitigation settings). This refers to distributions that might have negative values or sum to numbers other than 1 due to error mitigation post-processing (like Probabilistic Error Cancellation). 

However, in standard usage with current implementations (V2) and default settings, we largely focus on **sampled bitstrings** (counts) or **probability mass functions** (normalized counts) which approximate the true probability distribution of the quantum state.

## 2. Theory of Error Suppression & Mitigation

One of the main reasons to use the Sampler primitive over `backend.run()` is access to advanced error handling options. Two key techniques often exposed via Sampler options are **Dynamical Decoupling** and **Pauli Twirling**.

### A. Dynamical Decoupling (DD)

**The Problem**: Qubits lose their quantum information over time due to interaction with the environment (decoherence aka $T_1$ and $T_2$ times). This happens even when the qubit is "idle" (waiting for other qubits to finish their gates).

**The Theory**: Dynamical Decoupling applies a sequence of pulses (typically $\pi$-pulses, like $X$ gates) to the idle qubits. These pulses repeatedly flip the qubit state along opposite axes on the Bloch sphere (like a "spin echo"). 

*   **Analogy**: Imagine a runner on a track. If they drift slightly left due to wind, flipping them 180 degrees makes them drift right (relative to the track), cancelling out the original drift.

**In Qiskit**: You can enable this in the Sampler options, and the transpiler/scheduler will automatically insert these sequences into idle windows.

### B. Pauli Twirling

**The Problem**: Systematic, coherent errors (e.g., a gate always over-rotating by 1 degree) are very damaging because they can add up constructively ($1 + 1 + 1...$).

**The Theory**: Twirling wraps a noisy gate (like a CNOT) with random Pauli gates ($I, X, Y, Z$) that mathematically cancel out effectively to the identity operation *if there were no noise*. However, in the presence of noise, this randomization converts "coherent" errors (systematic shifts) into "incoherent" stochastic noise (random bit flips).

*   **Why do this?** Incoherent noise leads to the "depolarizing channel," which is much easier to model and mitigate than coherent errors. It turns a "bad" predictable error into "random" noise that averages out.

## 3. Practical Implementation

Now that we understand the theory, let's look at how to instantiate a Sampler (specifically **Sampler V2**) and configure these options. We will use `FakeManilaV2` to simulate a real backend environment locally.

In [1]:
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit_ibm_runtime import SamplerV2 as Sampler

# 1. Initialize the backend
# We use a fake backend which mimics the properties of the generic 'Manila' device
backend = FakeManilaV2()

# 2. Initialize the Sampler
# In V2, we can pass the backend directly to the mode argument for local simulation
sampler = Sampler(mode=backend)

In [2]:
# # Optional: Connect to Real Hardware
# # If you have IBM Quantum credentials configured, you can switch to a real backend.
# from qiskit_ibm_runtime import QiskitRuntimeService

# # 1. Save your account (Uncomment and run this ONCE if you haven't saved your token)
# # QiskitRuntimeService.save_account(channel="ibm_quantum", token="MY_IBM_QUANTUM_TOKEN", overwrite=True)

# try:
#     service = QiskitRuntimeService()
#     # Select the least busy 127-qubit device
#     real_backend = service.least_busy(
#         operational=True, simulator=False, min_num_qubits=127
#     )
#     backend = real_backend # Overwrite the fake backend
    
#     # Re-initialize Sampler with real backend
#     sampler = Sampler(mode=backend) 
#     print(f"Connected to real backend: {backend.name}")
# except Exception as e:
#     print("Qiskit Runtime Service unavailable or no credentials found.")
#     print("Using local FakeManilaV2 backend instead.")
#     print(f"Error Details: {e}")

### Configuring Error Suppression Options

We can now modify the `options` attribute of the sampler to enable the theoretical concepts discussed above. These options tell the Sampler how to process the circuit before execution.

In [3]:
# Enable Dynamical Decoupling
# 'XpXm' is a specific pulse sequence type consisting of X pulses
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XpXm"

# Enable Pauli Twirling for gates
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = 32 # How many random Pauli sets to generate per circuit
sampler.options.twirling.shots_per_randomization = 100 # Shots per set

### Running a Circuit with Sampler

Let's prepare a simple ISA (Instruction Set Architecture) circuit and run it. The Sampler requires transpiled circuits that match the backend's constraints.

In [4]:
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager

# Create a simple circuit (Bell State)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

# Transpile for the backend (Crucial for Sampler V2)
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_qc = pm.run(qc)

# Run the job
job = sampler.run([isa_qc])
print(f"Job ID: {job.job_id()}")

# Get results
result = job.result()

# Inspect results
# Sampler V2 returns a PubResult object. We access the data by the register name 'meas' (default for measure_all)
pub_result = result[0]
counts = pub_result.data.meas.get_counts()

print(f"Counts: {counts}")

Job ID: 9b83dd7c-e53e-4a96-baa9-15b22c45e89e
Counts: {'00': 465, '11': 500, '01': 21, '10': 38}


/Users/daai6ga1hou2/anaconda3/envs/qiskit_env/lib/python3.11/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:274: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XpXm'}, 'twirling': {'enable_gates': True, 'num_randomizations': 32, 'shots_per_randomization': 100}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


## Conclusion

In this notebook, we bridged the gap between theory and execution:

1.  **Sampler Purpose**: We learned it samples the output probability distribution ($|\langle k | \psi \rangle|^2$).
2.  **Theory of Options**: We explored how **Dynamical Decoupling** (refocusing idle qubits) and **Pauli Twirling** (randomizing coherent errors) work physically to improve result quality.
3.  **Execution**: We applied these settings using `SamplerV2` and `FakeManilaV2`.